### Load packages

In [1]:
import sys
sys.path.extend(['../src', '../instagram_crawler'])

from hashtag_cluster import Cluster

%load_ext autoreload
%autoreload 2

### Set hyperparameters

In [2]:
SAMPLE_SIZE = 10
MIN_COOCURRENCE_PROBABILITY = 0.01
DIR = './output/'
topics = ['travel', 'japan', 'taiwan', 'korea', 'china', 'singapore', 'australia', 'europe']

### Crawl posts and construct topic clusters

In [3]:
for topic in topics:
    max_iters = 5
    while max_iters > 0:
        max_iters -= 1
        try:
            print("Constructing cluster for: %s" % topic)
            cluster = Cluster(topic, 
                              min_coocurrence_probablity=MIN_COOCURRENCE_PROBABILITY, 
                              sample_size=SAMPLE_SIZE)
            filename = DIR + topic
            cluster.save_cluster(filename)
            break
        except:
            print("Error occured constructing cluster for: %s \n" % topic)
            continue

Constructing cluster for: travel


fetching: 33it [00:01, 28.73it/s]                                                                                                                                                                                 


Done. Fetched 10 posts.


fetching: 33it [00:01, 27.15it/s]                                                                                                                                                                                 


Done. Fetched 10 posts.


...

Done. Fetched 10 posts.


fetching: 33it [00:01, 23.84it/s]                                                                                                                                                                                 


Done. Fetched 10 posts.


fetching: 33it [00:01, 26.86it/s]                                                                                                                                                                                 


Done. Fetched 10 posts.


### Load Saved Clusters

In [9]:
for topic in topics:
    cluster = Cluster(topic, None, None, DIR + topic)
    print(topic)
    print(cluster.tags)
    print('----------------------------------------------------------------------')

travel
{'travelphotography', 'picoftheday', 'love', 'travelgram', 'beautiful', 'travel', 'wanderlust', 'instagood', 'landscape', 'nature', 'city', 'photography', 'photooftheday', 'travelling'}
----------------------------------------------------------------------
japan
{'travel', 'visitjapan', 'japan', 'ファインダー越しの私の世界', 'nature', 'japanesegirl', 'model', 'followme'}
----------------------------------------------------------------------
taiwan
{'台灣', '台湾', 'taiwan', '旅行', 'travel', 'taiwan1', 'taipei', 'iseetaiwan', 'iformosa', '写真', 'amazingtaiwan'}
----------------------------------------------------------------------
korea
{'いいね返し', '좋아요', '셀카', 'follow', 'daily', 'instagood', '인친', '얼스타그램', '맞팔', '좋아요반사', '데일리', '일상', '소통', '셀스타그램', 'ootd', '선팔하면맞팔', '맞팔해요', 'followme', '셀피', 'like4like', 'selfie', 'korea', 'outfit', 'fff', '팔로우', 'girl'}
----------------------------------------------------------------------
china
{'travel', 'china', 'nature'}
----------------------------------------